# Basic `pathpy` concepts

[Run notebook in Google Colab](https://colab.research.google.com/github/pathpy/pathpy/blob/master/doc/tutorial/network.ipynb)

`pathpy` has a user-friendly API that makes it easy to handle directed and undirected networks, networks with single and multiple edges, multi-layer networks or temporal networks. It also provides interactive HTML visualisations that can be directly displayed inside `jupyter` notebooks, which makes it particularly suitable for educational settings.

To get started with `pathpy` we first import `pathpy` and assign the local alias `pp`:

In [ ]:
pip install git+git://github.com/pathpy/pathpy.git

In [ ]:
import pathpy as pp
import numpy as np

If the `import` statement completes without error message, the installation was successful and we can now use `pathpy` to generate, analyse, and visualise networks. 

## Creating networks

For this purpose `pathpy` provides the `Network` class. Calling the constructor will return an instance that represents an empty network with no nodes and no links. By default, networks in `pathpy` are directed. If you want to create an undirected network you can pass the parameter `directed=False` in the constructor. 

Printing the `Network` object will give a short string summary which tells whether the network is directed or undirected, as well as the number of unique nodes and links.

In [ ]:
n1 = pp.Network()
print(n1)

A network is directed by default, but we can create an undirected network by passing the parameter `directed=False`.

In [ ]:
n2 = pp.Network(directed=False)
print(n2)

The examples above show that each network has a unique identifier. By default, this unique ID is derived from the hash value of the underlying python object, which allows you to quickly check whether two variables actually refer to the same network object. If you prefer to manage your own UIDs that are eadsier to remember, you can assign custom IDs by explicitly passing a uid property:

In [ ]:
n = pp.Network(directed=False, uid='MyUndirectedNetwork')
print(n)

The simplest way to add nodes and edges is to call the functions `add_node` and `add_edge`. In both cases, we can simply pass unique string identifiers of nodes, which will then be used as UIDs of the underlying node objects. To create an undirected network with three nodes and two edges, we can write: 

In [ ]:
n = pp.Network(directed=False, multiedges=True, uid='ExampleNetwork')
n.add_node('a')
n.add_node('b')
n.add_node('c')
n.add_edge('a', 'b')
n.add_edge('b', 'c')
print(n)

Unless we want to explicitly add isolated nodes with no incident edges, we can omit the explicit call of the `add_node` function. If we add edges any node that does not exist already will be created and added automatically. If we want to check explicitly whether a node exists before creating and edge, we can test this with the `in` operator on the set of node UIDS available via `Network.nodes.uids`:

In [ ]:
print('d' in n.nodes.uids)

The following code will automatically add a new node `d`, along with a new edge from `d` to `a`.

In [ ]:
n.add_edge('c', 'd')
n.add_edge('d', 'a')
print(n)

In [ ]:
print('d' in n.nodes.uids)

To count the number of nodes and edges in a network we can use the `number_of_nodes` and `number_of_edges` functions, or we could can compute `len` of `Network.nodes` and `Network.edges`:

In [ ]:
print('Network has {0} nodes and {1} edges'.format(n.number_of_nodes(), n.number_of_edges()))
print('Number of nodes: {0}'.format(len(n.nodes)))
print('Number of edges: {0}'.format(len(n.edges)))

### Node and Edge objects

In the simple example above, we generated nodes and edges by calling the `add_node` and `add_edge` function of the network instance. Internally, nodes and edges are represented as objects of type `Node` and `Edge` that can be referenced by one or more instances of type `Network`. Just like a `Network`, each instance of a `Node` and `Edge` has a UID. In the example above, `pathpy` has automatically created `Node` and `Edge` instances and has assigned the UIDs `a`, `b`, `c`, and `d` to those nodes. We can access those node objects via the node container `Network.nodes`. We can iterate through this dictionary to print a summary of all node objects referenced with a network object:

In [ ]:
for v in n.nodes:
    print(v)

We can also use the uid of a node to access a specific node object in a network by using the uid as an index to the `nodes` container:

In [ ]:
print(n.nodes['a'])

Importantly, the same node object can be added to multiple networks (which comes in handy if, for instance, we want to store that the same set of nodes is connected via different network topologies). Above, we have created a second (so far empty) undirected network. We can now add the node object with uid `a` to this network as follows:

In [ ]:
n2.add_node(n.nodes['a'])
print(n2)

While a node object can be added to multiple network objects, each network can contain only a single node with a given uid, so uncommenting the following line should raise an exception:

In [ ]:
# n.add_node('a')

Similar to `nodes`, the `edges` container of the network contains all edges of a network and each edge is actually stored as an `Edge` object. Let us iterate through the edges container of network `n` to better understand this:

In [ ]:
for e in n.edges:
    print('---')
    print(e)

We see that the edge container contains one `Edge` object instance for each edge that we added before. Each `Edge` has again a unique identifier, which has been automatically created in our example above. Just like for `Node` or `Network` objects, we can manually create an edge object with a custom UID that connects the two nodes `a` and `b` as follows:

In [ ]:
edge = pp.Edge(v=n.nodes['a'], w=n.nodes['b'], uid='MyEdge')
print(edge)

This `Edge` object has a different UID than the existing edge betwee node `a` and `b`, which is why we can add it to network `n` even though this network already contains an edge (with a different UID) between nodes `a` and `b`:

In [ ]:
n.add_edge(edge)
print(n)

The summary of the network confirms that the network now contains five edges (between four pairs of nodes). This native support for multi-edge networks is an important feature of `pathpy`. It also means that every pair of nodes can be connected by more than one edge. We can access those edges via the `Network.edges` container in multiple ways. First, we can simply iterate through the edge objects as shown before. Second, we can directly access an `Edge` with a given UID as follows:

In [ ]:
print(n.edges['MyEdge'])

Finally, we often want to access those edges that connect a specific pair of nodes. We can thus alternatively pass a pair of node uids as index to `Network.edges`. Since multiple edges between the same pair of nodes are possible, this generally returns a list of Edge objects, which - in the case of the node pair `a` and `b` - contains two different edge objects with different UIDs.

In [ ]:
print(n.edges['b', 'a'])

We can access the degrees of nodes, i.e. the number of other nodes to which a node is connected, via the `degrees()` function of the Network. The degrees() function gives the undirected degree (i.e. irrespective of the directionality of an edge), while the `indegrees()` and `outdegrees()` functions give the degrees in a directed network (i.e. to how many other nodes the edges of a node point of from how many other nodes edges point to the given node).

All of those functions return a dictionary that can be indexed via the unique node ids. In the case of a multi-edge network, the degree counts multiple edges to the same neighbor, which is why the degree of node `a` in the network above is 3:

In [ ]:
n.degrees()['a']

We can also remove nodes or edges. A network can contain isolated nodes (i.e. nodes with no incident edges), while it can (obviously) only contain edges between nodes that exist in the network. 

To ensure this constraint, a call to `remove_node` will also remove all edges that are incident to the removed node, i.e. if we remove node `a` the `Edge` object with uid `MyEdge` will be removed from the network (along with two additional edges (a,b) and (a,d)):

In [ ]:
n.remove_node('a')
print('MyEdge' in n.edges.uids)
print(n)

A call to `remove_edge` on the other hand does not remove the nodes incident to the removed edge, thus possibly leaving isolated nodes. If we remove the edge from `b` to `c` this will leave an isolated node `b`, which is still in the network:

In [ ]:
n.remove_edge('b', 'c')
print('b' in n.nodes.uids)
print(n.nodes['b'])
print(n)

Note that we can either remove a specific edge between a pair of nodes or all edges between a pair of nodes. To better understand this, let us create three different edges between two nodes 'x' and 'y' and add them to a network. To simplify the construction of networks, we can use the functions `add_nodes` and `add_edges` which add multiple nodes and edges at a time:

In [ ]:
n3 = pp.Network(multiedges=True)
n3.add_nodes('x', 'y')
e1 = pp.Edge(n3.nodes['x'], n3.nodes['y'], uid='edge1')
e2 = pp.Edge(n3.nodes['x'], n3.nodes['y'], uid='edge2')
e3 = pp.Edge(n3.nodes['x'], n3.nodes['y'], uid='edge3')
n3.add_edges(e1, e2, e3)
print(n3)

If we only remove the single edge with uid `edge2` between `x` and `y`, we can call:

In [ ]:
n3.remove_edge('edge2')
print(n3.edges['x', 'y'])

At the same time, we can remove all edges between `x` and `y` by calling:

In [ ]:
n3.remove_edge('x', 'y')
print(n3.edges['x', 'y'])

## Networks, Nodes and Edges with attributes

You may wonder why `pathpy` stores nodes and edges as objects rather than as simple strings or numbers. The reason is that we often want to use networks to model relational data that contains additional information on nodes, edges, or networks. To support this, all `pathpy` objects can store arbitrary key-value data in the form of attributes. Let's explore this in a toy example for a social network:

In [ ]:
trolls = pp.Network(uid='Trolls')
trolls.add_node('t')
trolls.add_node('b')
trolls.add_node('w')
trolls.add_edge('t', 'b', uid='t-b')
trolls.add_edge('t', 'w', uid='t-w')
print(trolls)

In [ ]:
trolls.nodes['t']['name'] = 'Tom'
trolls.nodes['t']['age'] = 156

trolls.nodes['b']['name'] = 'Bert'
trolls.nodes['b']['age'] = 96

trolls.nodes['w']['name'] = 'William'
trolls.nodes['w']['age'] = 323

Just like nodes, `Edge` objects can store arbitrary attributes that we can read and write via the `edges` dictionary in the `Network` class. We can store attributes as follows:

In [ ]:
trolls.edges['t-b']['strength'] = 2.0
trolls.edges['t-b']['type'] = 'like'

trolls.edges['t-w']['strength'] = 5.0
trolls.edges['t-w']['type'] = 'dislike'

In [ ]:
print(trolls.edges['t-b'].attributes)
print(trolls.edges['t-w'].attributes)

Numerical properties of edges are often used to store the strength or weight of interactions in a system. Such numerical property can also be considered in the degree calculation. Above, we have a directed network, so we can use the `indegrees()` and `outdegrees()` function to access the directed degrees of nodes. In our example, Tom has an outdegree of two since there are two directed edges to Bert and William:

In [ ]:
trolls.outdegrees()['t']

In [ ]:
trolls.outdegrees()['b']

To additionally consider numerical attributes in the degree calculation, we can use the weight parameter of the degrees(), indegrees() or outdegrees() functions. To calculate the total strength of all outgoing edges from Tom we can write:

In [ ]:
trolls.outdegrees(weight='strength')['t']

Finally, apart from node and edge attributes we can also store attributes at the network-level:

In [ ]:
trolls['chapter'] = 'Roast Mutton'
print(trolls.attributes)

Apart from using the attribute dictionary, we can directly assign attributes when we create nodes, edges, or networks. For this, we simply pass key/value parameters to the constuctor of the corresponding objects or to the `add_node` and `add_edge` functions. An equivalent way to create the network above would thus be:

In [ ]:
trolls = pp.Network(multiedges=True,name='Trolls', chapter='Roast Mutton')
tom = pp.Node(uid='t', name='Tom', age=156)
bert = pp.Node(uid='b', name='Bert', age=96)
william = pp.Node(uid='w', name='William', age=323)
e1 = pp.Edge(tom, bert, type='like', strength=2.0)
e2 = pp.Edge(tom, william, type='dislike', strength=5.0)
trolls.add_edge(e1)
trolls.add_edge(e2)
print('---\nNetwork attributes')
print(trolls.attributes)
print('---\nNode attributes')
for v in trolls.nodes:
    print(v, v.attributes)
print('---\nEdge attributes')
for e in trolls.edges:
    print(e, e.attributes)

## Adjacency matrices

As we will see in more detail next week, adjacency matrices are an important mathematical representation of networks. The topology of a matrix is represented in terms of a matrix A, where an entry A[i,j]=1 indicates that an edge exists from the i-th to the j-th node of the network. The absendce of edges is encoded by zero entries. The size of an adjacency matrix representation of a network with n nodes is generally $n^2$, which is not suitable for networks with thousands or millions of nodes. `pathpy` nevertheless supports efficient adjacency matrix calculation for *sparse* networks, i.e. networks where the majority of node pairs are not connected by an edge. Instead of a fully populated matrix, a call to `Network.adjacency_matrix()` returns a *sparse matrix object*, i.e. an efficient representation of the indices and values of non-zero entries:

In [ ]:
print(trolls.adjacency_matrix())

We use the `csr_matrix` of the sparse linear algebra module in scipy, which means we can directly apply matrix algebra operations to sparse adjacency matrices. If we instead want a dense matrix that includes zero entries, we can write:

In [ ]:
print(trolls.adjacency_matrix().todense())

The adjacency matrix entries above shows that two edges from Tom to Bert and from Tom to William exist. Moreover, the fact that the matrix is assymetric tells us that this is a directed network. By default, a binary matrix representation is returned where entries store the presence or absence of edges as 0 or 1 entries. If we want to use numerical attributes of edges instead, we can again pass the name of a numerical attribute that should be used as edge `weight`:

In [ ]:
print(trolls.adjacency_matrix(weight='strength').todense())

How does `pathpy` populate adjaecency matrices if the network contains multiple edges between the same pair of nodes? Let's try this by creating another edge between Tom and Bert, and let's further add a strength attribute:

In [ ]:
e3 = pp.Edge(tom, bert, type='sibling', strength=42.0)
trolls.add_edge(e3)
print(trolls)

If we now generate an adjacency matrix, the entries contain the *number of different edge objects* between pairs of nodes:

In [ ]:
trolls.adjacency_matrix().todense()

If we use a numerical attribute to calculate the matrix entries in such a network, the attributes of all edges between the same pair of nodes is automatically summed:

In [ ]:
trolls.adjacency_matrix(weight='strength').todense()